In [1]:
# This Python file uses the following encoding: utf-8
import os
import pickle
from PyQt6 import QtCore


def make_index(file):
    origins = []
    current_origin = 0

    original_position = file.tell()
    file.seek(0)

    while True:
        try:
            pickle.load(file)
            origins.append(current_origin)
            current_origin = file.tell()
        except EOFError:
            break
        except Exception as e:
            print(f"Exception: {e} in daqfile initializing index of file")

    #print(f"Make index {type(file)}")
    file.seek(original_position)

    return origins

class DaqFile(QtCore.QObject):
    # Provides appending and random access to pickled objects in a file

    def __init__(self, filename):
        super().__init__()
        name, ext = os.path.splitext(filename)
        if(ext != ".npy"):
            raise Exception(f"Daqfile: wrong file type: {ext}, must be a '.npy' file")
        self.filename = filename
        self.origins = None
        self.file = None

    def record(self):
        #does record require us to pickle??
        if(os.path.exists(self.filename)):
            print("Cannot overwrite an existing file")
            return
        self.file = open(self.filename, 'a+b')
        self.origins = make_index(self.file)

    def playback(self):
        try:
            self.file = open(self.filename, 'r+b')
        except Exception as e:
            print(f"Exception: {e} in daqfile for playback")
            raise

        self.origins = make_index(self.file)
        return True

    def count(self):
        if(self.origins is None):
            raise Exception("Daqfile: file has not been initialized")
        return len(self.origins)

    def get(self, n):
        self.file.seek(self.origins[n])
        return pickle.load(self.file)

    def append(self, obj):
        original_position = self.file.tell()

        self.file.seek(0, os.SEEK_END)
        self.origins.append(self.file.tell())
        pickle.dump(obj, self.file)

        self.file.seek(original_position)

    def seek(self, index):
        self.file.seek(self.origins[index])

    def next(self):
        # may throw EOFError
        return pickle.load(self.file)

    def close(self):
        self.file.close()
